In [22]:
import os
import winsound         # for sound  
import win32com.client
import pythoncom
import datetime, time# for sleep
import sys
import os.path
from os import path
import pandas as pd # command 로 시작할 때는 모듈 import 위치고려
from datetime import timedelta
import codecs
import shutil
from init_Stock import *

def __StandbyCall():
#     start_time = "09:00"
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Login_History"+_Name+".txt"
    print("# Standby Start : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))
    while datetime.datetime.today().strftime("%H:%M") < start_time : # ---> start_time보다 크면 while문 빠져나감, 시작 Call
        now_time = datetime.datetime.today().strftime("%H:%M:%S")
        
        if now_time > "08:55:00" : # 8시 55분보다 늦은 시간이면 print를 찍는다
            print("# Ready to Start ..... now %s" % (now_time))
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# Ready to Start ..... now %s\n" % (now_time))
            output.close() # ---> close 가 되어야 write 처리가 완료된다
            
        time.sleep(60)
        
    print("\n# Standby End : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))

def __Init_Conn() :

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Login_History"+_Name+".txt"
    class XASessionEventHandler:
        
        login_state = 0

        def OnLogin(self, code, msg):

            if code == "0000":
                print("로그인 성공")
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s ---> 로그인 성공 \n" % (datetime.datetime.today().strftime("%Y%m%d %H%M")))
                output.close() # ---> close 가 되어야 write 처리가 완료된다
                XASessionEventHandler.login_state = 1
            else:
                print("로그인 실패")
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s ---> 로그인 에러 \n" % (datetime.datetime.today().strftime("%Y%m%d %H%M")))
                output.close() # ---> close 가 되어야 write 처리가 완료된다

    instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)

    # 접속할 서버의 기본 주소는 'hts.ebestsec.co.kr'인데 모의 투자인 경우에는 'demo.ebestsec.co.kr'을 사용
    print("Server_Name : %s" % Server_Name)
    instXASession.ConnectServer(Server_Name, 20001)
#     instXASession.ConnectServer("hts.ebestsec.co.kr", 20001)
    instXASession.Login(id, passwd, cert_passwd, 0, 0)

    while XASessionEventHandler.login_state == 0:
        pythoncom.PumpWaitingMessages()

    num_account = instXASession.GetAccountListCount()
#     for i in range(num_account):
#         account = instXASession.GetAccountList(i)
#         print("계좌번호 : %s" % (account))

def __NAV_History(expcode) :

    Long_Term_Dic = {}

    ##################################################
    # 주식 현재가 호가 조회
    
    class XAQueryEventHandlerT1903:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT1903.query_state = 1

        def OnReceiveMessage(self, error, code, message):
            print("OnreceiveMessage", error, code, message)

    ##################################################
    instXAQueryT1903 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1903)
    instXAQueryT1903.ResFileName = "C:\\eBEST\\xingAPI\\Res\\T1903.res"
    instXAQueryT1903.SetFieldData("T1903InBlock", "shcode", 0, expcode)
    # instXAQueryT1903.SetFieldData("T1903InBlock", "date", 0, 20210511)

    instXAQueryT1903.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT1903.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

    count = instXAQueryT1903.GetBlockCount("t1903OutBlock1")

    grate_list = []
    for i in range(count):
        date = instXAQueryT1903.GetFieldData("T1903OutBlock1", "date", i)
        price = instXAQueryT1903.GetFieldData("T1903OutBlock1", "price", i) # 현재가
        sign = instXAQueryT1903.GetFieldData("T1903OutBlock1", "sign", i) # 전일대비 구분
        change = instXAQueryT1903.GetFieldData("T1903OutBlock1", "change", i) # 전일대비
        volume = instXAQueryT1903.GetFieldData("T1903OutBlock1", "volume", i) # 누적거래량
        navdiff = instXAQueryT1903.GetFieldData("T1903OutBlock1", "navdiff", i) # NAV대비
        nav = instXAQueryT1903.GetFieldData("T1903OutBlock1", "nav", i) # NAV ---> 현재 자산가치
        navchange = instXAQueryT1903.GetFieldData("T1903OutBlock1", "navchange", i) # 전일대비
        crate = instXAQueryT1903.GetFieldData("T1903OutBlock1", "crate", i) # 추적오차
        grate = instXAQueryT1903.GetFieldData("T1903OutBlock1", "grate", i) # 괴리
        jisu = instXAQueryT1903.GetFieldData("T1903OutBlock1", "jisu", i) # 지구
        jichange = instXAQueryT1903.GetFieldData("T1903OutBlock1", "jichange", i) # 전일대비
        jirate = instXAQueryT1903.GetFieldData("T1903OutBlock1", "jirate", i) # 전일대비율

        print(date, price, sign, change, volume, navdiff, navchange, crate, grate, jisu, jichange, jirate)
        nav = float(nav)
        price = float(price)
        # print(price, nav, ((price-nav)/price)*100)
        # print(price, nav, ((price-nav)/nav)*100)
        # print(price, nav, grate)
        grate_list.append(float(grate))

    return count, grate_list

def __NAV_Current(expcode) :

    Long_Term_Dic = {}

    ##################################################
    # 주식 현재가 호가 조회
    
    class XAQueryEventHandlerT1901:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT1901.query_state = 1

        def OnReceiveMessage(self, error, code, message):
            print("OnreceiveMessage", error, code, message)

    ##################################################
    instXAQueryT1901 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1901)
    instXAQueryT1901.ResFileName = "C:\\eBEST\\xingAPI\\Res\\T1901.res"
    instXAQueryT1901.SetFieldData("T1901InBlock", "shcode", 0, expcode)

    instXAQueryT1901.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT1901.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

    count = instXAQueryT1901.GetBlockCount("t1901OutBlock")

    print(count)
    # date_list, open_list, close_list, change_list, diff_list = [], [], [], [], []
    for i in range(count):
        hname = instXAQueryT1901.GetFieldData("T1901OutBlock", "hname", i)
        price = instXAQueryT1901.GetFieldData("T1901OutBlock", "price", i) # 현재가
        sign = instXAQueryT1901.GetFieldData("T1901OutBlock", "sign", i) # 전일대비 구분
        change = instXAQueryT1901.GetFieldData("T1901OutBlock", "change", i) # 전일대비
        diff = instXAQueryT1901.GetFieldData("T1901OutBlock", "diff", i) # 등락율
        volume = instXAQueryT1901.GetFieldData("T1901OutBlock", "volume", i) # 누적거래량
        recprice = instXAQueryT1901.GetFieldData("T1901OutBlock", "recprice", i) # 기준가
        high52w = instXAQueryT1901.GetFieldData("T1901OutBlock", "high52w", i) # 52주 최고가
        low52w = instXAQueryT1901.GetFieldData("T1901OutBlock", "low52w", i) # 52주 최저가
        per = instXAQueryT1901.GetFieldData("T1901OutBlock", "per", i) # 52주 최저가
        nav = instXAQueryT1901.GetFieldData("T1901OutBlock", "nav", i) # NAV ---> 현재 자산가치
        navdiff = instXAQueryT1901.GetFieldData("T1901OutBlock", "navdiff", i) # NAV대비
        navchange = instXAQueryT1901.GetFieldData("T1901OutBlock", "navchange", i) # 전일대비
        cocrate = instXAQueryT1901.GetFieldData("T1901OutBlock", "cocrate", i) # 추적오차율
        kasis = instXAQueryT1901.GetFieldData("T1901OutBlock", "kasis", i) # 괴리율

        print("hname : %s" % hname)
        print("현재가 : %s" % price)
        print("전일대비 구분 : %s" % sign)
        print("전일대비 : %s" % change)
        print("등락율 : %s" % diff)
        print("누적거래량 : %s" % volume)
        print("기준가 : %s" % recprice)
        print("52주 최고 : %s" % high52w)
        print("52주 최저 : %s" % low52w)
        print("PER : %s" % per)
        print("NAV : %s" % nav)
        print("NAV Diff : %s" % navdiff)
        print("전일대비 NAV : %s" % navchange)
        print("추적오차율 : %s" % cocrate)
        print("괴리율 : %s" % kasis)

if __name__ == '__main__' :
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    # File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"

    print("# Init Connection")
    __Init_Conn()

    expcode = "233740"
    count, grate_list = __NAV_History(expcode)

    print("# 과거 %s 일 동안 " % count)
    AVG_grate_list = sum(grate_list) / len(grate_list)
    print(min(grate_list))
    print(AVG_grate_list)
    print(max(grate_list))
    __NAV_Current(expcode)

# Init Connection
Server_Name : hts.ebestsec.co.kr
로그인 성공
OnreceiveMessage 0 00000 조회완료
20210512 13460 5 350 12525648 -71.13 370.11 1.26 -0.53 1365.03 19.41 -1.40
20210511 13810 5 675 11798345 -91.24 -638.65 5.47 -0.66 1384.44 30.74 2.27
20210510 14485 2 575 10042839 -54.89 542.36 5.47 -0.38 1415.18 27.69 2.00
20210507 13910 2 360 11435642 -87.53 395.81 5.47 -0.63 1387.49 18.31 1.34
20210506 13550 5 65 12396822 -51.72 -126.39 5.47 -0.38 1369.18 4.86 0.36
20210504 13615 2 210 24980593 -113.11 253.15 5.48 -0.82 1374.04 12.43 0.91
20210503 13405 5 795 18439637 -69.96 -861.04 5.49 -0.52 1361.61 43.86 3.33
20210430 14200 5 185 15629116 -136.00 -204.56 5.50 -0.95 1405.47 10.85 0.78
20210429 14385 5 380 14828401 -155.56 -385.02 5.51 -1.07 1416.32 19.40 1.39
20210428 14765 5 905 10057683 -160.58 -900.58 5.50 -1.08 1435.72 41.83 3.00
20210427 15670 5 405 6792937 -156.16 -456.24 5.51 -0.99 1477.55 19.95 1.37
20210426 16075 2 65 5601478 -207.40 92.89 5.52 -1.27 1497.50 3.41 0.23
20210423 16010 3 